# DESAFIO STONE:

## PROPOSTA:

Access the PostgreSQL database using the following credentials:

    host: db-stone.cjepwwjnksng.us-east-1.rds.amazonaws.com
    port: 5432
    database name: postgres
    user: read_only_user
    password: banking123


The database contains credit card transactional data in 4 tables:

    customers
    cards
    transactions
    frauds

Develop code to extract 2 kinds of datasets and export them to csv files. The required datasets are the following:

    1) The number of transactions and the total value purchased of each credit card grouped by card number and card family.
    2) All the customer ids that have "Diamond" segment and made at least 40 transactions.
   

Extras:

The following extra tasks are not required, but can give your solution bonus points.

    3) The table frauds shows all the transaction ids that were proven to be fraudulent. Analyze the data to find a correlation between the fraudulent transactions and the other features of the dataset. Explain your results.

    4) Develop a Dockerfile and/or a docker-compose file to automate your data processing application.
_______________________________________________________________________________________________________________________________

## SOLUÇÃO:

### - Importando as bibliotecas básicas e me conectando ao banco de dados:

Conectando à base de dados ao python...
Seguindo o tutorial  de https://www.postgresqltutorial.com/postgresql-python/connect/ :

1 - É preciso importar os módulos csv e psycopg2 (possui os métodos para fazer a conexão):

In [1]:
import csv
import psycopg2

In [2]:
import math  #será usado futuramente...

2 - É preciso criar uma variável da conexão:

In [3]:
con = psycopg2.connect(
                        host='db-stone.cjepwwjnksng.us-east-1.rds.amazonaws.com',
                        port=5432,
                        database='postgres',
                        user='read_only_user',
                        password='banking123')

3 - Também devemos criar uma variável para o "cursor", que fará a comunicação com o banco de dados.

In [4]:
cur = con.cursor()

### - Conhecendo as tabelas e seus relacionamentos:

In [5]:
#tupla com os nomes das tabelas:
nomes_tabelas=('customers','cards','transactions','frauds')

In [6]:
#conhecendo as colunas de cada tabela:
for t in nomes_tabelas:
    query_header = "SELECT column_name FROM information_schema.columns WHERE table_name = '"+t+"'"
    cur.execute(query_header)
    header = cur.fetchall()
    
    query_exemplo = "SELECT * FROM "+t+" LIMIT 1"
    cur.execute(query_exemplo)
    exemplo = cur.fetchone()
    print("%s:\n%s\n%s\n"% (t,header,exemplo))

customers:
[('id',), ('age',), ('segment',), ('vintage_group',)]
('CC25034', 35, 'Diamond', 'VG1')

cards:
[('card_number',), ('card_family',), ('credit_limit',), ('customer_id',)]
('8638-5407-3631-8196', 'Premium', 530000, 'CC67088')

transactions:
[('id',), ('card_number',), ('transaction_date',), ('value',), ('segment',)]
('CTID28830551', '1629-9566-3285-2123', datetime.date(2016, 4, 24), 23649, 'SEG25')

frauds:
[('transaction_id',), ('fraud_flag',)]
('CTID50558449', True)



In [7]:
#Decidi checar se havia diferença entre as colunas "segment", que aparecem nas tabelas "customers" e "transactions"
cur.execute('SELECT DISTINCT segment FROM customers')
row = cur.fetchall()
print("segments de 'customers':\n%s\n"%row)
cur.execute('SELECT DISTINCT segment FROM transactions')
row = cur.fetchall()
print("segments de 'transactions':\n%s" %row)

segments de 'customers':
[('Platinum',), ('Gold',), ('Diamond',)]

segments de 'transactions':
[('SEG16',), ('SEG20',), ('SEG19',), ('SEG23',), ('SEG17',), ('SEG15',), ('SEG24',), ('SEG22',), ('SEG18',), ('SEG11',), ('SEG13',), ('SEG12',), ('SEG14',), ('SEG25',), ('SEG21',)]


In [8]:
#Checando se as tabelas Frauds e Transactions tem o mesmo número de linhas...
cur.execute("SELECT COUNT(*) FROM transactions")
query1 = cur.fetchall()
cur.execute("SELECT COUNT(*) FROM frauds")
query2 = cur.fetchall()
lista = [query1[0][0],query2[0][0]]
print(lista)

[10000, 109]


In [9]:
#Confirmando que a tabela frauds só carrega as transações com fraudes...
cur.execute("SELECT DISTINCT fraud_flag FROM frauds")
tipos_de_fraude = cur.fetchall()
print(tipos_de_fraude)

[(True,)]


### Resolvendo os itens 1 e 2

In [10]:
#Resolução do item 1:
cur.execute("SELECT c.card_number,c.card_family,COUNT(t.id), SUM(t.value) FROM transactions t INNER JOIN cards c ON t.card_number = c.card_number GROUP BY c.card_number,c.card_family ORDER BY SUM(t.value) DESC")
item_1 = cur.fetchall()

#Salvando item A em .csv no arquivo "Item_1.csv"
saida = open("Item_1.csv",'w',newline = '')
escrever = csv.writer(saida)
escrever.writerow(("Card Number","Card Family","Transaction Number","Total Value Purchased"))
for i in item_1:
    escrever.writerow(i)
saida.close()

In [11]:
#Resolvendo o item 2:
cur.execute("SELECT cus.id,cus.segment,COUNT(tran.id) FROM transactions tran INNER JOIN cards car ON tran.card_number = car.card_number INNER JOIN customers cus ON car.customer_id = cus.id WHERE cus.segment = 'Diamond' GROUP BY cus.id,cus.segment HAVING COUNT(tran.id)>39 ORDER BY COUNT(tran.id) DESC")
item_2 = cur.fetchall()

#Salvando item 2 em .csv no arquivo "ItemB.csv"
saida = open("Item_2.csv",'w',newline = '')
escrever = csv.writer(saida)
escrever.writerow(("Custumer ID","Segment","Total Number of Transactions"))
for i in item_2:
    escrever.writerow(i)
saida.close()

## Extras:

### - Item 3:

Para resolver o item 3, a minha estratégia foi:
- Criar uma tabela que traga todas as transações, a indicação se a transação é fraudulenta e as variáveis que eu quero testar.
- Testar a dependência de fraud_flag em cada uma dessas variáveis, por usando o método do Qui-Quadrado.

    (Me baseei nos conhecimentos adquiridos em uma disciplina de simulação e nos tutoriais: "A Gentle Introduction to the Chi-Squared Test for Machine Learning" e "How to Choose a Feature Selection Method For Machine Learning" do site https://machinelearningmastery.com)

In [12]:
#Pegando a Tabela Desejada:
cur.execute("SELECT tran.id,transaction_date,value,tran.segment,fraud_flag,card_family,credit_limit,age,cus.segment,vintage_group FROM transactions tran INNER JOIN cards car ON tran.card_number = car.card_number INNER JOIN customers cus ON car.customer_id = cus.id LEFT JOIN frauds f on tran.id=f.transaction_id")
transacoes_expandida = cur.fetchall()
print("Exemplo de linha da tabela:\ntran.id, transaction_date, value, tran.segment, fraud_flag, card_family, credit_limit, age, cus.segment, vintage_group")
print(transacoes_expandida[0])

Exemplo de linha da tabela:
tran.id, transaction_date, value, tran.segment, fraud_flag, card_family, credit_limit, age, cus.segment, vintage_group
('CTID28830551', datetime.date(2016, 4, 24), 23649, 'SEG25', None, 'Platinum', 194000, 23, 'Gold', 'VG3')


In [13]:
# Transformando a Tabela Extraída em uma Lista de Listas(que podem ser modificadas mais facilmente):
lista = []
for i in transacoes_expandida: #Aqui eu decidi transformar os valores de "fraud_flag" em 1(é fraude) e 0(não é fraude). 
    k = list(i)
    if k[4]==True:
        k[4]=1
    else:
        k[4]=0
    lista.append(k)
    
conta_fraude = 0 #Fiz essa contagem para verificar se eu extrai corretamente todos as linhas.
conta_todo = 0   #O certo seria contar 109 fraudes e 10000 transações
for i in lista:
    conta_todo += 1
    if i[4]==1:
        conta_fraude += 1
print(conta_fraude,conta_todo)

109 10000


#### Testes Qui-Quadradro:

Nesta etapa, farei testes qui-quadrado da coluna "fraud_flag" com as demais colunas, uma de cada vez.
Esse teste estatístico permite checar a relação de dependência entre duas variáveis categóricas.

No nosso caso, as variáveis categóricas são:

    fraud_flag, tran.segment, card_family, cus.segment, vintage_group

Também temos três variáveis numéricas, que são:

    value, credit_limit, age

Irei começar tratando as variáveis numéricas, que podem ser transformadas em categorias pela regra de Sturges...

In [14]:
# Usando a regra de Sturges para definir o número de categorias (k)...
# k = 1+ 3.3*log(n); onde n é o tamanho da amostra(número de transações fraudulentas, calculado anteriormente)
tamanho_amostra = conta_fraude
k = math.ceil(1+3.3*math.log(tamanho_amostra,10))
print("Tamanho da amostra: %d\nNúmero de Categorias K: %d"%(tamanho_amostra,k))

Tamanho da amostra: 109
Número de Categorias K: 8


In [15]:
#Vamos começar avaliando a relação entre "frauds" e "values"...

#É preciso saber a amplitude do dados "values", ou seja: qual é a diferença entre o maior e o menor valor?
def lista_coluna(tabela,indice_col):
    l = []
    for i in tabela:
        l.append(i[indice_col])
    return l

def amplitude(vetor):
    a = (max(vetor) - min(vetor))
    print("Amplitude(a): %d"% a)
    return int(a)

coluna_values = lista_coluna(lista,2)

values_amplitude = amplitude(coluna_values)

# para a classificação, os valores serão classificados em "k" intervalos diferentes de tamanho "h":

h = values_amplitude/k
print("Tamanho do intervalo de classificação(h): %.2f"%h)

# Para ilustrar os intervalos criei a lista abaixo, mostra os limites superiores de cada um:
ini_class_values = []
for i in range(1,(k+1)):
    ini_class_values.append(int(i*h + min(coluna_values)))
print("\nLimites superiores dos intervalos de valores:\n%s"%ini_class_values) 

Amplitude(a): 49892
Tamanho do intervalo de classificação(h): 6236.50

Limites superiores dos intervalos de valores:
[6339, 12576, 18812, 25049, 31285, 37522, 43758, 49995]


In [16]:
#O próximo passo é criar a "Tabela Contingência" que irá contar as ocorências em cada classificação:
tabela_fraudesValues = [[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0]]
for i in lista:
    #descubro a posição da contagem de value:
    pos = min(int((i[2]-min(coluna_values))/h),(k-1))
    tabela_fraudesValues[i[4]][pos] += 1
print("\nTabela Contingência:\n",tabela_fraudesValues)


Tabela Contingência:
 [[1272, 1255, 1269, 1229, 1251, 1231, 1198, 1186], [7, 15, 14, 9, 14, 28, 9, 13]]


In [17]:
#Importo as bibliotecas em que o método Qui-Quadrado, já foi desenvolvido
from scipy.stats import chi2_contingency
from scipy.stats import chi2

In [18]:
#Declaro a função a seguir, que retorna a estatística calculada, o "p-valor",
# os graus de liberdade, a tabela de frequências esperada e já avalia se as variáveis são dependentes ou não:
def avalia_dependencia(tabela_contingencia):
    estat, p, gdl, esperada = chi2_contingency(tabela_contingencia)
    print('Graus de Liberdade = %d' % gdl)
    print("Tabelas Esperadas:\n",esperada)
    # interpretando o teste estatistico:
    prob = 0.95
    critica = chi2.ppf(prob, gdl)
    print('\nProbabilidade = %.3f, E. crítica=%.3f, Estatísca = %.3f' % (prob, critica, estat))
    if abs(estat) >= critica:
        print('Variáveis Dependentes (rejeita H0)')
    else:
        print('Variáveis Independentes (falha em rejeitar H0)')
    # interpretando o p-valor
    alpha = 1.0 - prob
    print('\nNível de Significância = %.3f, p=%.3f' % (alpha, p))
    if p <= alpha:
        print('Variáveis Dependentes (rejeita H0)')
    else:
        print('Variáveis Independentes (falha em rejeitar H0)')

In [19]:
avalia_dependencia(tabela_fraudesValues)

Graus de Liberdade = 7
Tabelas Esperadas:
 [[1265.0589 1256.157  1269.0153 1224.5058 1251.2115 1245.2769 1193.8437
  1185.9309]
 [  13.9411   13.843    13.9847   13.4942   13.7885   13.7231   13.1563
    13.0691]]

Probabilidade = 0.950, E. crítica=14.067, Estatísca = 21.453
Variáveis Dependentes (rejeita H0)

Nível de Significância = 0.050, p=0.003
Variáveis Dependentes (rejeita H0)


# ------------------------------------------------------------------------------------------
De forma análoga, as tabelas contingência de "frauds" com "credit limits" e "age" serão criadas e postas à prova:

### Analise de Fraudes x Limite de Crédito:

In [20]:
coluna_credLim = lista_coluna(lista,6)  #coluna de índice 6 na lista 
credLim_amplitude = amplitude(coluna_credLim)
h = credLim_amplitude/k
print("Tamanho do intervalo de classificação(h): %.2f"%h)

ini_class_CredLim = []
for i in range(1,(k+1)):
    ini_class_CredLim.append(int(i*h + min(coluna_credLim)))
print("\nLimites superiores dos intervalos de valores:\n%s"%ini_class_CredLim)

tabela_fraudesCredLim = [[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0]]
for i in lista:
    pos = min(int((i[6]-min(coluna_credLim))/h),(k-1))
    tabela_fraudesCredLim[i[4]][pos] += 1
print("\nTabela Contingência:\n",tabela_fraudesCredLim)

Amplitude(a): 897000
Tamanho do intervalo de classificação(h): 112125.00

Limites superiores dos intervalos de valores:
[114125, 226250, 338375, 450500, 562625, 674750, 786875, 899000]

Tabela Contingência:
 [[4423, 1969, 644, 564, 393, 676, 634, 588], [44, 26, 10, 9, 5, 4, 6, 5]]


In [21]:
avalia_dependencia(tabela_fraudesCredLim)

Graus de Liberdade = 7
Tabelas Esperadas:
 [[4.4183097e+03 1.9732545e+03 6.4687140e+02 5.6675430e+02 3.9366180e+02
  6.7258800e+02 6.3302400e+02 5.8653630e+02]
 [4.8690300e+01 2.1745500e+01 7.1286000e+00 6.2457000e+00 4.3382000e+00
  7.4120000e+00 6.9760000e+00 6.4637000e+00]]

Probabilidade = 0.950, E. crítica=14.067, Estatísca = 5.859
Variáveis Independentes (falha em rejeitar H0)

Nível de Significância = 0.050, p=0.556
Variáveis Independentes (falha em rejeitar H0)


### Análise de Fraudes x Idades:

In [22]:
coluna_idades = lista_coluna(lista,7)  #coluna de índice 7 na lista 
idades_amplitude = amplitude(coluna_idades)
h = idades_amplitude/k
print("Tamanho do intervalo de classificação(h): %.2f"%h)

ini_class_idades = []
for i in range(1,(k+1)):
    ini_class_idades.append(int(i*h + min(coluna_idades)))
print("\nLimites superiores dos intervalos de valores:\n%s"%ini_class_idades)

tabela_fraudesIdades = [[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0]]
for i in lista:
    pos = min(int((i[7]-min(coluna_idades))/h),(k-1))
    tabela_fraudesIdades[i[4]][pos] += 1
print("\nTabela Contingência:\n",tabela_fraudesIdades)

Amplitude(a): 30
Tamanho do intervalo de classificação(h): 3.75

Limites superiores dos intervalos de valores:
[23, 27, 31, 35, 38, 42, 46, 50]

Tabela Contingência:
 [[991, 1357, 1276, 1013, 1242, 1400, 1129, 1483], [12, 13, 17, 11, 13, 23, 6, 14]]


In [23]:
avalia_dependencia(tabela_fraudesIdades)

Graus de Liberdade = 7
Tabelas Esperadas:
 [[ 992.0673 1355.067  1278.9063 1012.8384 1241.3205 1407.4893 1122.6285
  1480.6827]
 [  10.9327   14.933    14.0937   11.1616   13.6795   15.5107   12.3715
    16.3173]]

Probabilidade = 0.950, E. crítica=14.067, Estatísca = 8.307
Variáveis Independentes (falha em rejeitar H0)

Nível de Significância = 0.050, p=0.306
Variáveis Independentes (falha em rejeitar H0)


### Agora os testes da função Qui-quadrada serão realizados com variáveis categóricas:

Primeiro será necessário encontrar para cada coluna de variável categórica suas classificações distintas.
Essas classificações serão 

tran.segment(3),card_family(5),cus.segment(8),vintage_group(9)

### Análise de Fraudes x Segmento da Transação:

In [24]:
cur.execute("SELECT distinct(tran.segment) FROM transactions tran")# Puxo categorias distintas da base de dados.
tranSegmentsDistintas = cur.fetchall()

aux = []                         #Gravo as categorias em uma lista
for i in tranSegmentsDistintas: 
    aux.append(list(i))
tranSegmentsDistintas = aux

print("tran.segment:\n",tranSegmentsDistintas)
k = len(tranSegmentsDistintas)   #k é o número de categorias

tran.segment:
 [['SEG16'], ['SEG20'], ['SEG19'], ['SEG23'], ['SEG17'], ['SEG15'], ['SEG24'], ['SEG22'], ['SEG18'], ['SEG11'], ['SEG13'], ['SEG12'], ['SEG14'], ['SEG25'], ['SEG21']]


In [25]:
tabela_fraudesTranSegments = [[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]] #Construo a tabela contingência
for i in lista:
    for j in range(0,k):                   #Adiciono +1 na posição equivalente de cada ocorrencia dentro da tabela contingêcia
        if i[3] == tranSegmentsDistintas[j][0]:
            tabela_fraudesTranSegments[i[4]][j] += 1                  
print("\nTabela Contingência:\n",tabela_fraudesTranSegments)


Tabela Contingência:
 [[659, 662, 655, 696, 643, 677, 661, 623, 678, 644, 675, 689, 617, 689, 623], [8, 6, 3, 12, 8, 6, 8, 9, 4, 15, 4, 6, 6, 4, 10]]


In [26]:
avalia_dependencia(tabela_fraudesTranSegments)

Graus de Liberdade = 14
Tabelas Esperadas:
 [[659.7297 660.7188 650.8278 700.2828 643.9041 675.5553 661.7079 625.1112
  674.5662 651.8169 671.5989 687.4245 616.2093 685.4463 626.1003]
 [  7.2703   7.2812   7.1722   7.7172   7.0959   7.4447   7.2921   6.8888
    7.4338   7.1831   7.4011   7.5755   6.7907   7.5537   6.8997]]

Probabilidade = 0.950, E. crítica=23.685, Estatísca = 21.590
Variáveis Independentes (falha em rejeitar H0)

Nível de Significância = 0.050, p=0.087
Variáveis Independentes (falha em rejeitar H0)


-------------------------------------------------------------------------------------------------
A partir de agora o mesmo processo vai se repetir para cada uma das outras variáveis categóricas.

### Análise de Fraudes x Segmento do Cliente:

In [27]:
cur.execute("SELECT distinct(cus.segment) FROM customers cus") 
cusSegmentsDistintas = cur.fetchall()

aux = []
for i in cusSegmentsDistintas: 
    aux.append(list(i))
cusSegmentsDistintas = aux

print("cus.segment:\n",cusSegmentsDistintas)
k = len(cusSegmentsDistintas)

cus.segment:
 [['Platinum'], ['Gold'], ['Diamond']]


In [28]:
tabela_fraudesCusSegments = [[0,0,0],[0,0,0]]
for i in lista:
    for j in range(0,k):
        if i[8] == cusSegmentsDistintas[j][0]:
            tabela_fraudesCusSegments[i[4]][j] += 1
print("\nTabela Contingência:\n",tabela_fraudesCusSegments)


Tabela Contingência:
 [[2260, 3225, 4406], [28, 27, 54]]


In [29]:
avalia_dependencia(tabela_fraudesCusSegments)

Graus de Liberdade = 2
Tabelas Esperadas:
 [[2263.0608 3216.5532 4411.386 ]
 [  24.9392   35.4468   48.614 ]]

Probabilidade = 0.950, E. crítica=5.991, Estatísca = 3.018
Variáveis Independentes (falha em rejeitar H0)

Nível de Significância = 0.050, p=0.221
Variáveis Independentes (falha em rejeitar H0)


### Análise de Fraudes x Família do Cartão:

In [30]:
cur.execute("SELECT distinct(card_family) FROM cards")
card_FamilyDistintas = cur.fetchall()

aux = []
for i in card_FamilyDistintas: 
    aux.append(list(i))
card_FamilyDistintas = aux

print("tran.segment:\n",card_FamilyDistintas)
k = len(card_FamilyDistintas)

tran.segment:
 [['Premium'], ['Platinum'], ['Gold']]


In [31]:
tabela_fraudesCardFamily = [[0,0,0],[0,0,0]]
for i in lista:
    for j in range(0,k):
        if i[5] == card_FamilyDistintas[j][0]:
            tabela_fraudesCardFamily[i[4]][j] += 1
print("\nTabela Contingência:\n",tabela_fraudesCardFamily)


Tabela Contingência:
 [[4054, 2240, 3597], [45, 25, 39]]


In [32]:
avalia_dependencia(tabela_fraudesCardFamily)

Graus de Liberdade = 2
Tabelas Esperadas:
 [[4054.3209 2240.3115 3596.3676]
 [  44.6791   24.6885   39.6324]]

Probabilidade = 0.950, E. crítica=5.991, Estatísca = 0.017
Variáveis Independentes (falha em rejeitar H0)

Nível de Significância = 0.050, p=0.992
Variáveis Independentes (falha em rejeitar H0)


### Análise de Fraudes x Vintage Group:

In [33]:
cur.execute("SELECT distinct(vintage_group) FROM customers")
vintage_groupDistintas = cur.fetchall()

aux = []
for i in vintage_groupDistintas: 
    aux.append(list(i))
vintage_groupDistintas = aux

print("tran.segment:\n",vintage_groupDistintas)
k = len(vintage_groupDistintas)

tran.segment:
 [['VG3'], ['VG2'], ['VG1']]


In [34]:
tabela_fraudesVintageGroup = [[0,0,0],[0,0,0]]
for i in lista:
    for j in range(0,k):
        if i[9] == vintage_groupDistintas[j][0]:
            tabela_fraudesVintageGroup[i[4]][j] += 1
print("\nTabela Contingência:\n",tabela_fraudesVintageGroup)


Tabela Contingência:
 [[3225, 2260, 4406], [27, 28, 54]]


In [35]:
avalia_dependencia(tabela_fraudesVintageGroup)

Graus de Liberdade = 2
Tabelas Esperadas:
 [[3216.5532 2263.0608 4411.386 ]
 [  35.4468   24.9392   48.614 ]]

Probabilidade = 0.950, E. crítica=5.991, Estatísca = 3.018
Variáveis Independentes (falha em rejeitar H0)

Nível de Significância = 0.050, p=0.221
Variáveis Independentes (falha em rejeitar H0)


### Conclusão:
#### - Pelo método Qui-Quadrado, apenas a coluna "value" mostrou uma relação de dependencia com a coluna "frauds"

In [36]:
#Encerra todas as conexões com o banco
cur.close()
con.close()